## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,65.80,56,76,3.62,broken clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,69.55,87,91,4.00,overcast clouds
2,2,Vao,NC,-22.6667,167.4833,70.27,62,30,16.53,scattered clouds
3,3,Georgetown,MY,5.4112,100.3354,85.93,79,20,3.00,few clouds
4,4,Sur,OM,22.5667,59.5289,82.36,48,9,5.08,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Georgetown,MY,5.4112,100.3354,85.93,79,20,3.00,few clouds
4,4,Sur,OM,22.5667,59.5289,82.36,48,9,5.08,clear sky
9,9,Kapaa,US,22.0752,-159.3190,85.98,74,55,5.01,broken clouds
12,12,Presidente Medici,BR,-11.1753,-61.9014,84.79,51,79,3.96,broken clouds
14,14,Sao Filipe,CV,14.8961,-24.4956,78.89,78,0,7.11,clear sky
24,24,Cayenne,GF,4.9333,-52.3333,78.84,89,100,0.00,overcast clouds
29,29,Vaini,TO,-21.2000,-175.2000,78.96,89,40,4.61,light intensity shower rain
30,30,Santa Cruz De Tenerife,ES,28.4682,-16.2546,75.65,84,75,3.00,broken clouds
39,39,Hithadhoo,MV,-0.6000,73.0833,82.54,70,93,10.92,overcast clouds
43,43,Agadez,NE,19.7500,10.2500,76.66,11,52,6.40,broken clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,85.93,few clouds,5.4112,100.3354,
4,Sur,OM,82.36,clear sky,22.5667,59.5289,
9,Kapaa,US,85.98,broken clouds,22.0752,-159.3190,
12,Presidente Medici,BR,84.79,broken clouds,-11.1753,-61.9014,
14,Sao Filipe,CV,78.89,clear sky,14.8961,-24.4956,
24,Cayenne,GF,78.84,overcast clouds,4.9333,-52.3333,
29,Vaini,TO,78.96,light intensity shower rain,-21.2000,-175.2000,
30,Santa Cruz De Tenerife,ES,75.65,broken clouds,28.4682,-16.2546,
39,Hithadhoo,MV,82.54,overcast clouds,-0.6000,73.0833,
43,Agadez,NE,76.66,broken clouds,19.7500,10.2500,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,85.93,few clouds,5.4112,100.3354,Cititel Penang
4,Sur,OM,82.36,clear sky,22.5667,59.5289,Sur Plaza Hotel
9,Kapaa,US,85.98,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Presidente Medici,BR,84.79,broken clouds,-11.1753,-61.9014,Oliveira Palace Hotel
14,Sao Filipe,CV,78.89,clear sky,14.8961,-24.4956,Hotel Xaguate
24,Cayenne,GF,78.84,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
29,Vaini,TO,78.96,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
30,Santa Cruz De Tenerife,ES,75.65,broken clouds,28.4682,-16.2546,Hotel Silken Atlántida Santa Cruz
39,Hithadhoo,MV,82.54,overcast clouds,-0.6000,73.0833,Scoop Guest House
46,Butaritari,KI,81.72,scattered clouds,3.0707,172.7902,Isles Sunset Lodge


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/VacationPy_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))